In [12]:
import os
import sys
import re
import glob
import torch
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from transformers import AutoModel, AutoTokenizer, BertTokenizer, BertForTokenClassification, Trainer, TrainingArguments


In [13]:
torch.cuda.is_available()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [14]:
# file_path = Path('/opt/ml/KLUE/klue_benchmark/klue-ner-v1.1/klue-ner-v1.1_train.tsv')
# raw_text = file_path.read_text().strip()
# raw_docs = re.split(r'\n\t?\n', raw_text)


In [15]:
file_list = []
for x in os.walk('/opt/ml/NER'):
    for y in glob.glob(os.path.join(x[0], '*_NER.txt')):    # ner.*, *_NER.txt
        file_list.append(y)

In [16]:
def read_file(file_list): # 이중 엔터를 기준으로 document 로써 구분을 하게 됨. 각 라인을 읽어나가면서 토큰들을 문장으로 붙여나가는 과정을 거칠 것임. 
    token_docs = []
    tag_docs = []
    for file_path in file_list:
        # print("read file from ", file_path)
        file_path = Path(file_path)
        raw_text = file_path.read_text().strip()
        raw_docs = re.split(r'\n\t?\n', raw_text)
        for doc in raw_docs:
            tokens = []
            tags = []
            for line in doc.split('\n'):
                if line[0:1] == "$" or line[0:1] == ";" or line[0:2] == "##":
                    continue
                try:
                    token = line.split('\t')[0]
                    tag = line.split('\t')[3]   # 2: pos, 3: ner
                    for i, syllable in enumerate(token):    # 음절 단위로 잘라서 (형태소 단위 -> 음절 단위로 바꿔주기 위함)
                        tokens.append(syllable) # 음절 단위 정보를 가져와서 문장을 만들어 감.  
                        modi_tag = tag
                        if i > 0:
                            if tag[0] == 'B':
                                modi_tag = 'I' + tag[1:]    # BIO tag를 부착할게요 :-) (잘라진 음절에 대해서도 BIO 태크를 붙이는 과정을 거치게 됨)
                        tags.append(modi_tag)
                except:
                    print(line)
            token_docs.append(tokens)
            tag_docs.append(tags)

    return token_docs, tag_docs

In [17]:
texts, tags = read_file(file_list[:])
print(len(texts))
print(len(tags))

19263
19263


In [18]:
texts, tags

([['초',
   '등',
   '학',
   '생',
   '인',
   '_',
   '두',
   '_',
   '자',
   '녀',
   '를',
   '_',
   '챙',
   '기',
   '기',
   '_',
   '위',
   '해',
   '_',
   '퇴',
   '근',
   '을',
   '_',
   '서',
   '두',
   '르',
   '면',
   '서',
   '_',
   '김',
   '밥',
   '으',
   '로',
   '_',
   '저',
   '녁',
   '을',
   '_',
   '때',
   '운',
   '_',
   '적',
   '이',
   '_',
   '많',
   '기',
   '는',
   '_',
   '했',
   '지',
   '만',
   '_',
   '‘',
   '후',
   '진',
   '국',
   '병',
   '’',
   '이',
   '라',
   '는',
   '_',
   '결',
   '핵',
   '을',
   '_',
   '앓',
   '게',
   '_',
   '될',
   '_',
   '줄',
   '은',
   '_',
   '상',
   '상',
   '도',
   '_',
   '못',
   '_',
   '했',
   '다',
   '.'],
  ['나',
   '물',
   '_',
   '건',
   '강',
   '하',
   '게',
   '_',
   '먹',
   '는',
   '_',
   '요',
   '리',
   '법',
   '은',
   '?',
   '…',
   '영',
   '양',
   '_',
   '살',
   '리',
   '는',
   '_',
   '나',
   '물',
   '_',
   '요',
   '리',
   '법',
   '_',
   "'",
   '나',
   '물',
   '_',
   '건',
   '강',
   '하',
   '게',
   '_',
   '먹',
   '는',

In [19]:
unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [20]:
for i, tag in enumerate(sorted(unique_tags)):
    print(tag)
# 출처 : https://stellarway.tistory.com/29
# 우리가 해야 하는 것 : 이름, 전화번호, 이메일주소, 직책
# 우리 task에 맞는 domain-special NE가 필요함.

# DAT 날짜
# DUR 기간
# LOC 지명
# MNY 통화
# NOH 기타 수량표현
# ORG 기관
# PER 인물
# PNT 비율
# POH 기타
# TIM 시간

B-DAT
B-DUR
B-LOC
B-MNY
B-NOH
B-ORG
B-PER
B-PNT
B-POH
B-TIM
I-DAT
I-DUR
I-LOC
I-MNY
I-NOH
I-ORG
I-PER
I-PNT
I-POH
I-TIM
O


In [21]:
train_texts, test_texts, train_tags, test_tags = train_test_split(texts, tags, test_size=.2)

In [22]:
MODEL_NAME = "klue/bert-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [23]:
pad_token_id = tokenizer.pad_token_id # 0 # 스페셜 토큰의 경우 레이블이 데이터셋에 존재하지 않아 따로 추가해주는..?
cls_token_id = tokenizer.cls_token_id # 101
sep_token_id = tokenizer.sep_token_id # 102
pad_token_label_id = tag2id['O']    # tag2id['O']
cls_token_label_id = tag2id['O']
sep_token_label_id = tag2id['O']

In [24]:
def ner_tokenizer(sent, max_seq_length):    
    pre_syllable = "_"
    input_ids = [pad_token_id] * (max_seq_length - 1)
    attention_mask = [0] * (max_seq_length - 1)
    token_type_ids = [0] * max_seq_length
    sent = sent[:max_seq_length-2]

    for i, syllable in enumerate(sent):
        if syllable == '_':
            pre_syllable = syllable
        if pre_syllable != "_":
            syllable = '##' + syllable  # 중간 음절에는 모두 prefix를 붙입니다.
            # 이순신은 조선 -> [이, ##순, ##신, ##은, 조, ##선]
        pre_syllable = syllable

        input_ids[i] = (tokenizer.convert_tokens_to_ids(syllable))
        attention_mask[i] = 1
    
    input_ids = [cls_token_id] + input_ids
    input_ids[len(sent)+1] = sep_token_id
    attention_mask = [1] + attention_mask
    attention_mask[len(sent)+1] = 1
    return {"input_ids":input_ids, # 기존의 토크나이저가 반환하는 것과 동일한 형태로 반환함.
            "attention_mask":attention_mask,
            "token_type_ids":token_type_ids}

In [25]:
tokenized_train_sentences = []
tokenized_test_sentences = []
for text in train_texts:    # 전체 데이터를 tokenizing 합니다.
    tokenized_train_sentences.append(ner_tokenizer(text, 128))
for text in test_texts:
    tokenized_test_sentences.append(ner_tokenizer(text, 128))

In [26]:
def encode_tags(tags, max_seq_length):
    # label 역시 입력 token과 개수를 맞춰줍니다 :-) (truncation, padding 과정이 들어가야.. )
    tags = tags[:max_seq_length-2]
    labels = [tag2id[tag] for tag in tags]
    labels = [tag2id['O']] + labels

    padding_length = max_seq_length - len(labels)
    labels = labels + ([pad_token_label_id] * padding_length)

    return labels

In [27]:
train_labels = []
test_labels = []
for tag in train_tags:
    train_labels.append(encode_tags(tag, 128))
for tag in test_tags:
    test_labels.append(encode_tags(tag, 128))

In [28]:
class TokenDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val) for key, val in self.encodings[idx].items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TokenDataset(tokenized_train_sentences, train_labels)
test_dataset = TokenDataset(tokenized_test_sentences, test_labels)

In [29]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,
    learning_rate=3e-5,
    save_total_limit=5
)

In [30]:
model = BertForTokenClassification.from_pretrained(MODEL_NAME, num_labels=len(unique_tags)) # 모델 초기화. num_lables : 우리가 구분해야 하는 태그의 개수를 명확하게 지정하는..
model.to(device)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset            # evaluation dataset
)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the mo

In [31]:
trainer.train()

Step,Training Loss
100,0.598000
200,0.316300
300,0.266000
400,0.218600
500,0.223500
600,0.199800
700,0.218900
800,0.208200
900,0.214100
1000,0.176800


TrainOutput(global_step=9635, training_loss=0.09785895514624318, metrics={'train_runtime': 1184.2117, 'train_samples_per_second': 8.136, 'total_flos': 6511722640934400.0, 'epoch': 5.0, 'init_mem_cpu_alloc_delta': 11255808, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 21852160, 'train_mem_gpu_alloc_delta': 1352506368, 'train_mem_cpu_peaked_delta': 196468736, 'train_mem_gpu_peaked_delta': 899638272})

In [32]:
def ner_inference(text) : # 학습된 모델을 가지고 추론을 진행해보자.
  
    model.eval()
    text = text.replace(' ', '_')

    predictions , true_labels = [], []
    
    tokenized_sent = ner_tokenizer(text, len(text)+2)
    input_ids = torch.tensor(tokenized_sent['input_ids']).unsqueeze(0).to(device)
    attention_mask = torch.tensor(tokenized_sent['attention_mask']).unsqueeze(0).to(device)
    token_type_ids = torch.tensor(tokenized_sent['token_type_ids']).unsqueeze(0).to(device)    
    
    with torch.no_grad():
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)
        
    logits = outputs['logits']
    logits = logits.detach().cpu().numpy()
    label_ids = token_type_ids.cpu().numpy()

    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    true_labels.append(label_ids)

    pred_tags = [list(tag2id.keys())[p_i] for p in predictions for p_i in p]

    print('{}\t{}'.format("TOKEN", "TAG"))
    print("===========")
    # for token, tag in zip(tokenizer.decode(tokenized_sent['input_ids']), pred_tags):
    #   print("{:^5}\t{:^5}".format(token, tag))
    for i, tag in enumerate(pred_tags):
        print("{:^5}\t{:^5}".format(tokenizer.convert_ids_to_tokens(tokenized_sent['input_ids'][i]), tag))

In [33]:
text = '차투차 CHATOCHA'
ner_inference(text)

TOKEN	TAG
[CLS]	  O  
  차  	  O  
 ##투 	  O  
 ##차 	  O  
  _  	  O  
  C  	B-POH
 ##H 	I-POH
 ##A 	I-POH
 ##T 	I-POH
 ##O 	I-POH
 ##C 	I-POH
 ##H 	I-POH
 ##A 	I-POH
[SEP]	  O  


In [34]:
text = '금융부분이사  박원주'
ner_inference(text)

TOKEN	TAG
[CLS]	  O  
  금  	  O  
 ##융 	  O  
 ##부 	  O  
 ##분 	  O  
 ##이 	  O  
 ##사 	  O  
  _  	  O  
  _  	  O  
  박  	B-PER
 ##원 	I-PER
 ##주 	I-PER
[SEP]	  O  


In [63]:
text = '본사 경기도 수원시 권선구 세화로 44 (평동)'
ner_inference(text)

TOKEN	TAG
[CLS]	  O  
  본  	  O  
 ##사 	  O  
  _  	  O  
  경  	B-LOC
 ##기 	I-LOC
 ##도 	I-LOC
  _  	  O  
  수  	B-LOC
 ##원 	I-LOC
 ##시 	I-LOC
  _  	  O  
  권  	B-LOC
 ##선 	I-LOC
 ##구 	I-LOC
  _  	  O  
  세  	B-LOC
 ##화 	I-LOC
 ##로 	I-LOC
  _  	I-LOC
  4  	I-LOC
 ##4 	I-LOC
  _  	  O  
  (  	  O  
 ##평 	B-LOC
 ##동 	I-LOC
[UNK]	  O  
[SEP]	  O  


In [66]:
text = '수원지점 경기도 수원시 권선구 권선로 308-5 도이치월드 219호'
ner_inference(text)

TOKEN	TAG
[CLS]	  O  
  수  	B-LOC
 ##원 	I-LOC
 ##지 	  O  
 ##점 	  O  
  _  	  O  
  경  	B-LOC
 ##기 	I-LOC
 ##도 	I-LOC
  _  	  O  
  수  	B-LOC
 ##원 	I-LOC
 ##시 	I-LOC
  _  	  O  
  권  	B-LOC
 ##선 	I-LOC
 ##구 	I-LOC
  _  	  O  
  권  	B-LOC
 ##선 	I-LOC
 ##로 	I-LOC
  _  	I-LOC
  3  	I-LOC
 ##0 	I-LOC
 ##8 	I-LOC
[UNK]	I-LOC
 ##5 	I-LOC
  _  	I-LOC
  도  	I-LOC
 ##이 	I-LOC
 ##치 	I-LOC
 ##월 	I-LOC
 ##드 	I-LOC
  _  	I-LOC
  2  	I-LOC
 ##1 	I-LOC
 ##9 	I-LOC
 ##호 	I-LOC
[SEP]	  O  


In [67]:
text = 'M 010 9121 5238 T 1644 8333'
ner_inference(text)

TOKEN	TAG
[CLS]	  O  
  M  	B-POH
  _  	I-POH
  0  	I-POH
 ##1 	I-POH
 ##0 	I-POH
  _  	I-POH
  9  	I-POH
 ##1 	I-POH
 ##2 	I-POH
 ##1 	I-POH
  _  	I-POH
  5  	I-POH
 ##2 	I-POH
 ##3 	I-POH
 ##8 	I-POH
  _  	  O  
  T  	B-POH
  _  	I-POH
  1  	I-POH
 ##6 	I-POH
 ##4 	I-POH
 ##4 	I-POH
  _  	I-POH
  8  	I-POH
 ##3 	I-POH
 ##3 	I-POH
 ##3 	I-POH
[SEP]	I-POH


In [68]:
text = 'E wjp3020@gmail.com F 031 224 4587'
ner_inference(text)

TOKEN	TAG
[CLS]	  O  
  E  	B-POH
  _  	I-POH
  w  	I-POH
 ##j 	I-POH
 ##p 	I-POH
 ##3 	I-POH
 ##0 	I-POH
 ##2 	I-POH
 ##0 	I-POH
[UNK]	I-POH
 ##g 	I-POH
 ##m 	I-POH
 ##a 	I-POH
 ##i 	I-POH
 ##l 	I-POH
[UNK]	I-POH
 ##c 	I-POH
 ##o 	I-POH
 ##m 	I-POH
  _  	  O  
  F  	B-POH
  _  	I-POH
  0  	I-POH
 ##3 	I-POH
 ##1 	I-POH
  _  	I-POH
  2  	I-POH
 ##2 	I-POH
 ##4 	I-POH
  _  	I-POH
  4  	I-POH
 ##5 	I-POH
 ##8 	I-POH
 ##7 	I-POH
[SEP]	I-POH


In [69]:
text = '차투차 CHATOCHA 금융부분이사 본사 경기도 수원시 권선구 세화로 44 (평동) 박원주 수원지점 경기도 수원시 권선구 권선로 308-5 도이치월드 219호 M 010 9121 5238 T 1644 8333 E wjp3020@gmail.com F 031 224 4587'
ner_inference(text)

TOKEN	TAG
[CLS]	  O  
  차  	  O  
 ##투 	  O  
 ##차 	  O  
  _  	  O  
  C  	B-ORG
 ##H 	I-ORG
 ##A 	I-ORG
 ##T 	I-ORG
 ##O 	I-ORG
 ##C 	I-ORG
 ##H 	I-ORG
 ##A 	I-ORG
  _  	  O  
  금  	  O  
 ##융 	  O  
 ##부 	  O  
 ##분 	  O  
 ##이 	  O  
 ##사 	  O  
  _  	  O  
  본  	  O  
 ##사 	  O  
  _  	  O  
  경  	B-LOC
 ##기 	I-LOC
 ##도 	I-LOC
  _  	  O  
  수  	B-LOC
 ##원 	I-LOC
 ##시 	I-LOC
  _  	  O  
  권  	B-LOC
 ##선 	I-LOC
 ##구 	I-LOC
  _  	  O  
  세  	B-LOC
 ##화 	I-LOC
 ##로 	I-LOC
  _  	I-LOC
  4  	I-LOC
 ##4 	I-LOC
  _  	  O  
  (  	  O  
 ##평 	B-LOC
 ##동 	I-LOC
[UNK]	  O  
  _  	  O  
  박  	B-PER
 ##원 	I-PER
 ##주 	I-PER
  _  	  O  
  수  	  O  
 ##원 	  O  
 ##지 	  O  
 ##점 	  O  
  _  	  O  
  경  	B-LOC
 ##기 	I-LOC
 ##도 	I-LOC
  _  	  O  
  수  	B-LOC
 ##원 	I-LOC
 ##시 	I-LOC
  _  	  O  
  권  	B-LOC
 ##선 	I-LOC
 ##구 	I-LOC
  _  	  O  
  권  	B-LOC
 ##선 	I-LOC
 ##로 	I-LOC
  _  	I-LOC
  3  	I-LOC
 ##0 	I-LOC
 ##8 	I-LOC
[UNK]	I-LOC
 ##5 	I-LOC
  _  	  O  
  도  	B-LOC
 ##이 	I-LOC
 ##치 	I-LOC
 ##월 	

In [71]:
text = '차투차 CHATOCHA 금융부분이사 박원주 본사 경기도 수원시 권선구 세화로 44 (평동) 수원지점 경기도 수원시 권선구 권선로 308-5 도이치월드 219호 M 010 9121 5238 T 1644 8333 E wjp3020@gmail.com F 031 224 4587'
ner_inference(text)

TOKEN	TAG
[CLS]	  O  
  차  	  O  
 ##투 	  O  
 ##차 	  O  
  _  	  O  
  C  	B-ORG
 ##H 	I-ORG
 ##A 	I-ORG
 ##T 	I-ORG
 ##O 	I-ORG
 ##C 	I-ORG
 ##H 	I-ORG
 ##A 	I-ORG
  _  	  O  
  금  	  O  
 ##융 	  O  
 ##부 	  O  
 ##분 	  O  
 ##이 	  O  
 ##사 	  O  
  _  	  O  
  박  	B-PER
 ##원 	I-PER
 ##주 	I-PER
  _  	  O  
  본  	  O  
 ##사 	  O  
  _  	  O  
  경  	B-LOC
 ##기 	I-LOC
 ##도 	I-LOC
  _  	  O  
  수  	B-LOC
 ##원 	I-LOC
 ##시 	I-LOC
  _  	  O  
  권  	B-LOC
 ##선 	I-LOC
 ##구 	I-LOC
  _  	  O  
  세  	B-LOC
 ##화 	I-LOC
 ##로 	I-LOC
  _  	I-LOC
  4  	I-LOC
 ##4 	I-LOC
  _  	  O  
  (  	  O  
 ##평 	B-LOC
 ##동 	I-LOC
[UNK]	  O  
  _  	  O  
  수  	  O  
 ##원 	  O  
 ##지 	  O  
 ##점 	  O  
  _  	  O  
  경  	B-LOC
 ##기 	I-LOC
 ##도 	I-LOC
  _  	  O  
  수  	B-LOC
 ##원 	I-LOC
 ##시 	I-LOC
  _  	  O  
  권  	B-LOC
 ##선 	I-LOC
 ##구 	I-LOC
  _  	  O  
  권  	B-LOC
 ##선 	I-LOC
 ##로 	I-LOC
  _  	I-LOC
  3  	I-LOC
 ##0 	I-LOC
 ##8 	I-LOC
[UNK]	I-LOC
 ##5 	I-LOC
  _  	  O  
  도  	B-LOC
 ##이 	I-LOC
 ##치 	I-LOC
 ##월 	